El siguiente paso es extraer información detallada de los 10 principales actores de cada una de las películas extraidas en la fase 1 utilizando Selenium (desde la página e IMDB). Debereis solo tener en cuenta los 10 primeros actores de cada película o corto (en caso de que los haya)La información que deberás extraer es:

Nombre.

Año nacimiento.

¿Por qué es conocido?

¿Qué hace?

Premios.

NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:

In [ ]:
[("Adam Sadler", 1966, ["El aguador", "Little Nicky", "Un papá genial", "El chico ideal" ], ["Reparto", "Producción", "Guion"] , ["American Comedy Awards, USA", "Annie Awards"],   ),  
...]

In [7]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  


# Recuperamos la lista final de tuplas

In [13]:
# Traer resultado tuplas total extraídos en la Fase 1
df = pd.read_csv('resultado_tuplas_total.csv', header=None)
lista_tuplas = [tuple(row) for row in df.values]

In [14]:
lista_tuplas = lista_tuplas[1:]

In [15]:
lista_tuplas[:10]

[('Movie', 'The A-Team', 2010.0, 6.0, 'tt0429493'),
 ('Movie', 'Segurança Nacional', 2010.0, 5.0, 'tt0439801'),
 ('Movie', 'Komaram Puli', 2010.0, 9.0, 'tt0464162'),
 ('Movie', 'Prince of Persia: The Sands of Time', 2010.0, 5.0, 'tt0473075'),
 ('Movie', 'Unstoppable', 2010.0, 12.0, 'tt0477080'),
 ('Movie', 'The Losers', 2010.0, 4.0, 'tt0480255'),
 ('Movie', 'Clash of the Titans', 2010.0, 4.0, 'tt0800320'),
 ('Movie', 'Cross the Line', 2010.0, 11.0, 'tt0810815'),
 ('Short', 'Red Princess Blues', 2010.0, nan, 'tt0903012'),
 ('Movie', 'Westbrick Murders', 2010.0, 5.0, 'tt0903036')]

In [16]:
# Bucle for para realizar pruebas
for peli in lista_tuplas[:]: # Prueba: Iterando para extraer el id_imdb y el nombre 
    id_imdb = peli[-1]
    nombre = peli[1]
    print(id_imdb)
    print(nombre)
    break

tt0429493
The A-Team


In [17]:
# Creación de la variables que vamos a usar en la extración con Selenium
url_imdb = "https://www.imdb.com/"
selector_cookies_imdb = "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"

In [18]:
# Creacción de las listas donde vamos a almacenar los resultados de la extración.
resultados_completos = []
resultados_parciales = [] # Nos permitimos añadir tuplas con hasta 2 valores nulos.
not_found = []

In [19]:
# Creamos una función para abrir el navegador 
def abrir_navegador(driver, url, selector):
    driver = webdriver.Chrome() # Inicializa el navegador Chrome
    sleep(3)
    driver.maximize_window() # Maximiza la ventana
    driver.get(url) # Navega a la url pasada por parámetro
    sleep(3)
    try:
        driver.find_element("css selector", selector).click() # Intentamos aceptar las cookies
        sleep(3)
    except: # Si salta error, imprime un mensaje avisando.
        print("No ha sido posible aceptar las cookies, puede que no saltó el aviso o que se haya cambiado el selector.")
    
    return driver # retorna el driver que será usado para las demás tareas

In [109]:
# Listas de selectores CSS para hacer iteraciones

selector_nombre_que_hace = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div"]
selector_anio_nasc = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > section > aside > div > span:nth-child(2)"]
selector_conocido_por = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-1c5b3162-0.fFgRFS > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid"]


selector_awards_click = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li:nth-child(2) > a"]
# Hay que hacer .click() en el selector awards y luego driver.back()

selector_awards = "#jump-to"



# Selectores de actores
select_actor1 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(2) > td:nth-child(2) > a"]
select_actor2 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(3) > td:nth-child(2) > a"]
select_actor3 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(4) > td:nth-child(2) > a"]
select_actor4 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(5) > td:nth-child(2) > a"]
select_actor5 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(6) > td:nth-child(2) > a"]
select_actor6 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(7) > td:nth-child(2) > a"]
select_actor7 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(8) > td:nth-child(2) > a"]
select_actor8 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(9) > td:nth-child(2) > a"]
select_actor9 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(10) > td:nth-child(2) > a"]
select_actor10 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(11) > td:nth-child(2) > a"]
lista_select_actores = [select_actor1, select_actor2, select_actor3, select_actor4, select_actor5, select_actor6, select_actor7, select_actor8, select_actor9, select_actor10]


In [82]:
# Creamos una función para iterar por los selectores haciendo los try except para extraer datos de imdb
def extrae_datos(lista_selector, driver):
    for selector in lista_selector:
        try:
            dato = driver.find_element("css selector", selector).text
            sleep(2)
            return dato 
        except Exception as e:
            excepcion = e

    error = f"No se ha podido extraer el dato, error de tipo: {excepcion}."
    not_found.append(error)
    dato = None
    return dato

In [111]:
def clicar(lista_selector, driver):
    for selector in lista_selector:
        try:
            driver.find_element("css selector", selector).click()
            return True
        except Exception as e:
            print(f"No ha sido posible clicar. Error:{e}.")
            return False

In [112]:
# Llamamos a todas las funciones para extraer los datos

driver1 = None
driver1 = abrir_navegador(driver1, url_imdb, selector_cookies_imdb)

contador = 0 # Creamos un contador para saber por que iteración vamos

'''
Maíra: 0:50, 200:400 ok, next = 400:1000
Yami: 50:100
Laura: 100:150
Carolina: 150:200
'''

# Iterar por los ids_imdbs para pasarlos por el campo search de la web de imdb:
for peli in lista_tuplas[:5]:
    id_imdb = peli[-1]

    # Pasamos el id_imdb al buscador de imdb
    driver1.find_element("css selector", "#suggestion-search").send_keys(id_imdb, Keys.ENTER)
    sleep(5)

    # Pinchamos en cast & crew
    driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li:nth-child(1) > a").click()
    sleep(3)

    for select_actor in lista_select_actores:    
        # Pinchamos en el actor
        pinchar_actor = clicar(select_actor, driver1)
        sleep(3)

        # Extraímos los datos de imdb
        nombre_que_hace = extrae_datos(selector_nombre_que_hace, driver1)
        nombre = nombre_que_hace.split('\n')[0]
        que_hace = nombre_que_hace.split('\n')[1:] # que_hace = extrae_datos(selector_nombre_que_hace, driver1)
        anio_nasc = extrae_datos(selector_anio_nasc, driver1)[-4:]
        conocido_por = extrae_datos(selector_conocido_por, driver1).split('\n')[::4]

        # Pinchamos en los awards
        pinchar_awards = clicar(selector_awards_click, driver1)
        sleep(6)
        premios = extrae_datos(selector_awards, driver1)

        break
        sleep(2)
        driver1.back()
        sleep(3)
        driver1.back()
        sleep(3)

        
    break















    # texto_actores = driver1.find_element("css selector", "#fullcredits_content").text
    # lista_texto_actores = texto_actores.split('\n')
    # lista_10_actores = []
    # lista_10_actores_limpio = []
    # for elemento in lista_texto_actores:
    #     elemento_cast = elemento
    #     cast = elemento.startswith('Cast')
    #     if cast:
    #         indice_cast = lista_texto_actores.index(elemento_cast)
    #         lista_10_actores.extend(lista_texto_actores[indice_cast+1:indice_cast+11])
    #         break

    # for actor in lista_10_actores:
    #     actor_limpio = actor.split(' ... ')[0]
    #     lista_10_actores_limpio.append(actor_limpio)
    
    # print(lista_10_actores_limpio)




    # # Creamos la tupla y la añadimos a la lista que corresponda (completa o sin tomatometer)
    # if tomatometer:
    #     tupla = imdb_rating, tomatometer, direccion, guionista, argumento, genero, duracion, nombre
    #     if None in tupla:
    #         contador += 1
    #         print(f"La tupla de la peli {nombre} no está completa: {tupla} Vamos por la iteración: {contador}")
    #     else:
    #         resultados_completos.append(tupla)
    #         contador += 1
    #         print(f"Hemos añadido la peli {nombre} a la lista completa. Vamos por la iteración: {contador}.")
    # else:
    #     tupla = imdb_rating, direccion, guionista, argumento, genero, duracion, nombre
    #     if None in tupla:
    #         contador += 1
    #         print(f"La tupla de la peli {nombre} no está completa: {tupla} Vamos por la iteración: {contador}")
    #     else:
    #         tupla = imdb_rating, tomatometer, direccion, guionista, argumento, genero, duracion, nombre
    #         resultados_sin_tomatometer.append(tupla)
    #         contador += 1
    #         print(f"Hemos añadido la peli {nombre} a la lista sin tomatometer. Vamos por la iteración: {contador}.")

driver1.close()

In [88]:
nombre.split('\n')[1:]

['Actor', 'Producer', 'Soundtrack']

In [86]:
nombre

'Liam Neeson\nActor\nProducer\nSoundtrack'

In [113]:
nombre

'Liam Neeson'

In [114]:
que_hace

['Actor', 'Producer', 'Soundtrack']

In [115]:
anio_nasc

'1952'

In [117]:
conocido_por

['Kinsey', "Schindler's List", 'The Grey', 'The A-Team']

In [119]:
premios

''